In [118]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [119]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [120]:
num_cars = 100
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())
print(data.collect().limit(5))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: "None"
shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 16.090781 ┆ -103.424082 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:19:03.511771 ┆ 16.090781 ┆ -103.424082 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:05:29.341110 ┆ 16.090781 ┆ -103.424082 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:30:00        ┆ 16.092839 ┆ -103.428635 ┆ null          │
│ car_0  ┆ 2023-01-01T00:54:31.785450 ┆ 16.092839 

## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [121]:
#data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime))
#print(data.collect())

data = data.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")
    .alias("timestamp")
)

print(data.collect())

data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))
print(data.collect())


shape: (632_138, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00.000000 ┆ 16.090781 ┆ -103.424082 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:19:03.511771 ┆ 16.090781 ┆ -103.424082 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:05:29.341110 ┆ 16.090781 ┆ -103.424082 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:30:00.000000 ┆ 16.092839 ┆ -103.428635 ┆ null          │
│ car_0  ┆ 2023-01-01T00:54:31.785450 ┆ 16.092839 ┆ -103.428635 ┆ null          │
│ …      ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_99 ┆ 2023-01-07T22:46:06.273499 ┆ 20.794662 ┆ -105.75214  ┆ tire_pressur

/tmp/ipykernel_11240/95882026.py:13: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))


shape: (632_138, 5)
┌────────┬───────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                     ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                           ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ datetime[ns]                  ┆ f64       ┆ f64         ┆ str           │
╞════════╪═══════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00           ┆ 16.090781 ┆ -103.424082 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:19:03.000511771 ┆ 16.090781 ┆ -103.424082 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:05:29.000341110 ┆ 16.090781 ┆ -103.424082 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:30:00           ┆ 16.092839 ┆ -103.428635 ┆ null          │
│ car_0  ┆ 2023-01-01 00:54:31.000785450 ┆ 16.092839 ┆ -103.428635 ┆ null          │
│ …      ┆ …                             ┆ …         ┆ …           ┆ …             │
│ car_99 ┆ 2023-01-07 22:46:06.000273499 ┆ 20

### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [122]:
#A function to order data by id_car and timestamp
def order_data(data):
    data = data.sort("car_id", "timestamp")
    return data

data = order_data(data)
print(data.collect())

agrupcion = data.group_by("car_id", maintain_order=True).agg(pl.all(), pl.col("timestamp").shift(1).alias("previous_timestamp"))
agrupacion.explode()
print(agrupcion.collect().limit(5))

shape: (632_138, 5)
┌────────┬───────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                     ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                           ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ datetime[ns]                  ┆ f64       ┆ f64         ┆ str           │
╞════════╪═══════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00           ┆ 16.090781 ┆ -103.424082 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:05:29.000341110 ┆ 16.090781 ┆ -103.424082 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:19:03.000511771 ┆ 16.090781 ┆ -103.424082 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:30:00           ┆ 16.092839 ┆ -103.428635 ┆ null          │
│ car_0  ┆ 2023-01-01 00:34:28.000067839 ┆ 16.092839 ┆ -103.428635 ┆ engine_check  │
│ …      ┆ …                             ┆ …         ┆ …           ┆ …             │
│ car_99 ┆ 2023-01-07 22:46:06.000273499 ┆ 20

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`
